In [ ]:
import networkx as nx
import os
import time
import random
import pandas as pd
import numpy as np
from networkx.algorithms.community import kernighan_lin_bisection

In [ ]:
def bfs(visited, queue, graph, node):
    visited.append(node)
    queue.append(node)
    count = 1

    while queue:
        m = queue.pop(0)

        for neighbour in graph.neighbors(m):
            if neighbour not in visited:
                count = count + 1
                visited.append(neighbour)
                queue.append(neighbour)
    return count

def bfs_count(G):
    visited = []
    queue = []

    return bfs(visited, queue, G, np.asarray(G.nodes)[0])

def binary_search(arr, x):
  low = 0
  high = len(arr) - 1
  mid = 0

  while low <= high:

    mid = (high + low) // 2

    if arr[mid] < x:
      low = mid + 1
    elif arr[mid] > x:
      high = mid - 1
    else:
      return mid
  return -1


def find_edges(graph, p1, p2):

  edge_list = np.array(graph.edges)
  node_list = np.array(graph.nodes)

  g1 = nx.Graph()
  g2 = nx.Graph()

  for edge in edge_list:
    if (edge[0] in p1) and (edge[1] in p1):
      g1.add_edge(edge[0], edge[1])

  for edge in edge_list:
    if (edge[0] in p2) and (edge[1] in p2):
      g2.add_edge(edge[0], edge[1])

  return (g1, g2)

def partition(G, partition=None, max_iter=10):
  blocks = kernighan_lin_bisection(G, partition=partition, max_iter=max_iter)
  p1 = np.array(list(blocks[0]))
  p2 = np.array(list(blocks[1]))

  return find_edges(G, p1, p2)

def isValid(g1, g2):
  c1 = bfs_count(g1)
  c2 = bfs_count(g2)

  if c1 != len(g1.nodes) or c2 != len(g2.nodes):
    return False
  return True

def labelize(G, max_iter):
  g1, g2 = partition(G, max_iter=max_iter)
  V = len(G.nodes)
  if round(V / 2 - V * 0.01) > len(g1.nodes) or round(V / 2 + V * 0.01) < len(g1.nodes):
    return False
  return isValid(g1, g2)


In [ ]:
random.seed(time.time() % 60)

def generate_graphs(model, num_graphs, graph_size):
    #m, k, p ve d değişkenler; bunları değiştirerek üretin
    #m=Number of edges to attach from a new node to existing nodes
    #n=Number of nodes in graph
    #k=Each node is connected to k nearest neighbors
    #p=Probability of edge existence
    #d=Degree of each node
    for i in range(num_graphs):
        max_iter = 10
        if graph_size > 500:
            max_iter = round(0.4*graph_size*math.log(graph_size, 10))
        probability = random.random() / 10 + 0.2
        k = random.randint(1, 5)
        # Generate a random graph based on the specified model
        if model == "erdos_renyi":
            G = nx.erdos_renyi_graph(graph_size, probability)
        elif model == "barabasi_albert":
            m = 2  # Define m before using it
            G = nx.barabasi_albert_graph(graph_size, m)
        elif model == "watts_strogatz":
            k = 2  # Define k before using it
            G = nx.watts_strogatz_graph(graph_size, k, probability)
        elif model == "random_regular":
            d = 4  # Define d before using it
            G = nx.random_regular_graph(d, graph_size)
        elif model == "random_geometric":
            G = nx.random_geometric_graph(graph_size, probability)
        elif model == "trees":
            G = nx.random_tree(graph_size, seed=random.randint(1, 100))
        elif model == "euler_graph":
            G = nx.eulerize(nx.complete_graph(graph_size))
        elif model == "hamiltonian_graph":
            G = nx.cycle_graph(graph_size)
        elif model == "bipartite_graph":
            G = nx.complete_bipartite_graph(graph_size//2, graph_size//2)

        label = labelize(G, max_iter)
        file_name = f"graphs_data/{model}_graph_{i + 1}_size{n}_label_{label}.dat"
        with open(file_name, "w") as file:
            for edge in nx.to_edgelist(G):
                file.write(f"{edge[0]} {edge[1]}\n")

if not os.path.exists('graphs_data'):
    os.makedirs('graphs_data')

# Parameters
num_graphs_per_model = 20
#bu da değişebilir her modelin graph sayısı
size = [100, 300, 500, 800, 1000, 2000, 4000, 6000]
models = ["erdos_renyi", "barabasi_albert", "watts_strogatz", "random_regular", "random_geometric", "trees", "euler_graph", "hamiltonian_graph", "bipartite_graph"]

for n in size:
    for model in models:
      print(model, n)
      generate_graphs(model, num_graphs_per_model, n)

